In [1]:

DATA_PATH = "data/CA-Final"


In [2]:
import numpy as np
import os
import sys
from scgt import GeoTiff

In [3]:
sys.path.append(os.path.join(os.getcwd(), "birdmaps"))
from birdmaps import bird_runs


## Bird Run Definition

In [4]:
birds_for_paper = []

bird_run = bird_runs.BirdRun(DATA_PATH)

birds = []

NUM_SIMULATIONS = 400
RUN_NAME = "Paper"

birds.append(bird_run.get_bird_run(
    "acowoo", "Acorn Woodpecker",
    do_validation=False, run_name=RUN_NAME,
    hop_distance=2, num_spreads=20,
    num_simulations=NUM_SIMULATIONS))

# Steller's Jay, hop distance = 2, num spreads = 4
birds.append(bird_run.get_bird_run(
    "stejay", "Steller's Jay",
    do_validation=False, run_name=RUN_NAME,
    hop_distance=2, num_spreads=3,
    num_simulations=NUM_SIMULATIONS))

In [5]:

california = GeoTiff.from_file(os.path.join(DATA_PATH, "Terrain", "california.tif"))
cal_tile = california.get_all_as_tile()

/Users/luca/venvs/venv37/lib/python3.7/site-packages/rasterio/__init__.py:223: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  path, mode, driver=driver, sharing=sharing, **kwargs


In [6]:
def compute_fch(bird):
    hab = GeoTiff.from_file(bird.habitat_fn)
    rep = GeoTiff.from_file(bird.repopulation_fn)
    hab_tile = hab.get_all_as_tile()
    rep_tile = rep.get_all_as_tile()
    habitat = np.sum(hab_tile.m * cal_tile.m) * 0.3 * 0.3 
    fca = np.sum(rep_tile.m * cal_tile.m / 255.) * 0.3 * 0.3
    connected_fraction = np.sum(rep_tile.m * cal_tile.m / 255.) / np.sum(hab_tile.m * cal_tile.m)
    print("{} habitat: {} FCH: {} connected fraction: {}".format(bird.name, habitat, fca, connected_fraction))
    return fca

In [7]:
for bird in birds:
    compute_fch(bird)

Acorn Woodpecker habitat: 66988.43999999999 FCH: 50667.97552941175 connected fraction: 0.7563689426028096
Steller's Jay habitat: 77716.43999999999 FCH: 68915.41835294111 connected fraction: 0.8867546989149415


Let us estimate the percentage difference from using 400 vs. 10000 simulations for Acorn Woodpecker (for Steller's Jay it's much lower).

In [10]:
fch10000 = compute_fch(birds[0])

acowoo400 = bird_run.get_bird_run(
    "acowoo", "Acorn Woodpecker",
    do_validation=False, run_name="Paper10000",
    hop_distance=2, num_spreads=20,
    num_simulations=10000)
fch400 = compute_fch(acowoo400)

100 * abs(fch10000 - fch400) / min(fch10000, fch400)

Acorn Woodpecker habitat: 66988.43999999999 FCH: 50667.97552941175 connected fraction: 0.7563689426028096
Acorn Woodpecker habitat: 66988.43999999999 FCH: 50708.97423529399 connected fraction: 0.7569809691835485


0.08091640815299239